# Course Project - Shor's Algorithm

CS251 Quantum Information Science, 2018 @ ACM Honors Class, SJTU


## Lecturer

Prof. **Runyao Duan**


## Group Member

- **Zetian Jiang** (516030910577)
- **Zhanghao Wu** (516030910593)
- **Zhou Fan** (516030910574)


## Abstract

We implemented Shor's Algorithm in this project. The quantum part of the algorithm, i.e. the phase estimation subroutine, is implemented in Q# programming language, and we encapsulated this part to provide a flexible callable interface. In this way one can easily call our quantum subroutine in programming languages other than Q# or C#, for instance, Python. With this cross-language feature, we were able to implement the classical part of Shor's algorithm in Python, which is presented later in this Jupyter notebook. In addition, we introduced parallel computation in the implementation of Shors' algorithm, running multiple order finding subroutine simultaneously to accelerate the process. Finally, we wrote a test script to factorize random numbers from 1 to 70, and the implementation passed the test.


## Adavanced Feature

Our implementation of Shor's algorithm has the following two advanced features described in the course task documentation:

- **Cross-language interoperability**: one can call our Q# subroutine in languages other than C#, such as Python
- **Parallel computation**: we introduced parallel computation in the implementation of Shor's algorithm


## Directory Structure 

...


## Process of Shor's Algorithm

The input is a number $N$ to be factorized.
1. Check if $N$ is a prime number, this can be done with some classical algorithm such as Miller-Rabin primality test. If $N$ is prime number, the factorization is no longer needed.
2. Check if $N$ is a power of a prime number, that is, if there exists a prime number $a$ such that $N = a^k$. If so, then $a$ is a factor of $N$, then we have the factorization of $N$.
3. Now the number $N$ can be represented as $$N = p_1^{k_1} p_2^{k_2} ... p_n^{k_n}.$$ Perform the following operations until we find a factorization of $N$:
   1. Generate a random number $x < N$.
   2. Check if $x$ is coprime to $N$, if not, then the greatest common divisor of $x$ and $N$ is a factor of $N$. Then we have a factorization of $N$.
   3. Now we know that $x$ and $N$ are coprime to each other. There exists a order $r$ of $x$ such that $$x^r \equiv 1 \pmod N,$$ and this order $r$ can be computed using quantum algorithm for phase estimation. This is the quantum part of Shor's algorithm.
   4. If the order $r$ is odd, go back to A to generate a new $x$. Else we have $$(x^{\frac{r}{2}} + 1)(x^{\frac{r}{2}} - 1) = x^r - 1 \equiv 0 \pmod N.$$ If $x^{\frac{r}{2}} + 1 \pmod N$ and $x^{\frac{r}{2}} - 1 \pmod N$ are both not zero, then we have a factorization of $N$. If not, go back to A to try a new random $x$.


## Cross-language Interoperability

To encapsulate the quantum subroutine implemented in Q#, we separate the Q# operations as a standalone .NET standard library project. And we compiled this project to generate the library assemblies. Then one can use operations in that library in languages other than C# by adding a reference to the assemblies file. For instance, to use our Q# library in Python, one could use the Python.NET package to load the assemblies, and one should also add references to the assemblies from the Q# development kit released by Microsoft as dependencies.

The following example shows how to load our assemblies in Python. One should first have Python.NET package installed in Python environment in order to use the clr module. One can use the command `pip install pythonnet` to install Python.NET package. It is also needed to add the path to the assemblies file in the system path such that the `clr` module could find the assemblies. Another requirement is Python library `qsharp`, which provides a QuantumSimulator class which wraps the Microsoft.Quantum.Simulation.Simulators.QuantumSimulator .NET class provided with the Quantum Development Kit. This wrapper provides a few useful convienence features that we will use later.

In [5]:
import sys
sys.path.append('./myShorLib/bin/Release/netstandard2.0')

In [6]:
import qsharp

In [8]:
import clr
clr.AddReference("Microsoft.Quantum.Canon")
clr.AddReference('myShorLib')

Here `myShorLib` is the quantum library we implemented in Q#. Having added reference to the assemblies, we can now use operations in the library. To call the operation, we could just import it just as a normal Python package.

## Parallel computation

In Shor's algorithm, we use the quantum phase estimation subroutine to find the order $r$ of a random number $x < N$, i.e. a positive number $r$ such that
$$x^r \equiv 1 \pmod N$$
We introduced parallel computation in this process, that is, performing the order finding subroutine for multiple random numbers simultaneously with multi-threading technique. This could accelerate the process of Shor's algorithm when $N$ is large. With the help of the cross-language interoperability of our implementation of the phase estimation subroutine, we can easily implement the parallel part in Python.